# Laboratorio 2 - Detección de SPAM

## Nombre: Josué Sagastume
Carné: 18173

Curso: Security Data Science


---



In [1]:
!pip install contractions
!pip install textsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#importacion de librerias

import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn import feature_extraction, tree, model_selection, metrics
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
%matplotlib inline
from yellowbrick.features import Rank2D
from yellowbrick.features import RadViz
from sklearn.model_selection import train_test_split
from urllib.parse import urlparse
from pandas_profiling import ProfileReport
import nltk
import contractions
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True



---


# Parte 1 - Ingeniería de Características

## Exploración de Datos

In [4]:
df1 = pd.read_csv('completeSpamAssassin.csv')
print(df1.shape)
df1.sample(n=5).head() # print a random sample of the DataFrame

(6046, 3)


,Unnamed: 0,Body,Label
3220,3220,"Daniel Quinlan wrote:\n> Exactly, except that...",0
5519,5519,"Once upon a time, Matthias wrote :> On Thu, 18...",0
2509,2509,Robert Harley writes:\n> Chuck Murcko wrote:> ...,0
3794,3794,URL: http://www.aaronsw.com/weblog/000616\nDat...,0
656,656,\nÂ Â \nDear \n Homeown...,1


In [5]:
print(df1['Label'].value_counts())

0    4150
1    1896
Name: Label, dtype: int64


In [6]:
df2 = pd.read_csv('enronSpamSubset.csv')
print(df2.shape)
df2.sample(n=5).head() # print a random sample of the DataFrame

(10000, 4)


,Unnamed: 0,Unnamed: 0.1,Body,Label
1224,10864,10864,Subject: importante\n hola .\n crea un sitio w...,1
2749,1850,1850,"Subject: re : aekdju , the cat raised\n our us...",1
8725,23166,23166,"Subject: monterrey , mexico\n ms . kitchen -\n...",0
552,4727,4727,Subject: watch and learn about it merle\n stro...,1
2593,5643,5643,Subject: = ? iso - 8859 - 7 ? q ? get _ cial =...,1


In [7]:
print(df2['Label'].value_counts())

1    5000
0    5000
Name: Label, dtype: int64


In [8]:
df_merged = pd.merge(df1, df2, how='outer')

In [9]:
df_merged.sample(n=5).head()

,Unnamed: 0,Body,Label,Unnamed: 0.1
3926,3926,"URL: http://www.newsisfree.com/click/-5,829818...",0,NaN
8920,8312,Subject: is his son still in pain\n stop hurti...,1,8312.0
11800,32508,Subject: re : organisation\n did not get a con...,0,32508.0
6727,4708,Subject: extra power . don ' t think . just ac...,1,4708.0
14590,32355,Subject: updated q & as for enron employees\n ...,0,32355.0


In [10]:
df_merged = df_merged.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [11]:
df_merged = df_merged.rename(columns={"Body": "body", "Label": "status"})
df_merged.sample(n=5).head()

,body,status
4308,"URL: http://www.newsisfree.com/click/-3,870478...",0
7858,Subject: re : good time\n new offshore pharmac...,1
4113,"URL: http://www.newsisfree.com/click/215,8,215...",0
2809,Why am I so passionate about decentralization?...,0
15681,"Subject: fw : tw 1090 meeting\n fyi ,\n adr\n ...",0


In [12]:
print(df_merged['status'].value_counts())

0    9150
1    6896
Name: status, dtype: int64


## Preprocesamiento

In [13]:
import unicodedata
import re

df_merged['body'] = df_merged['body'].str.strip()
df_merged['body'] = df_merged['body'].str.lower()

def removerAcentos(texto):
    texto = unicodedata.normalize('NFKD', str(texto)).encode('ascii','ignore').decode('utf-8','ignore')
    return texto

def removerCaracteresEspecialesNumerosSimbolos(texto, removerDigitos =False):
    patron = r'[^a-zA-Z0-9\s]' if not removerDigitos else r'[^a-zA-Z\s]'
    texto = re.sub(patron,'', texto, re.I|re.A)
    return texto

In [14]:
df_merged['body'] = df_merged['body'].apply(removerAcentos)
df_merged['body'] = df_merged['body'].apply(removerCaracteresEspecialesNumerosSimbolos)
df_merged['body'] = df_merged['body'].apply(contractions.fix)
stop_words_english = nltk.corpus.stopwords.words('english')

In [15]:
def doc(texto):
    tokens = nltk.word_tokenize(texto)
    filtered_tokens = [token for token in tokens if token not in stop_words_english]
    texto = ' '.join(filtered_tokens)
    return texto

df_merged['body'] = df_merged['body'].apply(doc)
df_merged.sample(n=5)

,body,status
9482,subject red light tickets rrvnt introducing ph...,1
1098,new web technology unlimited web conferencing ...,1
15264,subject draft 6 th floor layout attached first...,0
14569,subject hpl optimization team mentioned hpl st...,0
13915,subject request closed access request john and...,0


## Representación de Texto

In [16]:
cv = CountVectorizer(min_df=0.1, max_df=0.9)
cv_matrix = cv.fit_transform(df_merged['body'])
cv_matrix = cv_matrix.toarray()
cv_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 2, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [17]:
# Palabras únicas del corpus
vocabulario = cv.get_feature_names()
# Mostrar el vector
df_mergedFinal = pd.DataFrame(cv_matrix, columns=vocabulario)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [18]:
df_mergedFinal.sample(n=5)

,10,2000,2002,also,best,business,cc,click,com,company,...,thanks,time,today,us,use,want,way,well,work,would
5686,0,0,0,0,0,0,0,0,0,0,...,1,2,0,0,0,0,0,0,0,1
2388,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
3990,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12511,1,0,0,0,0,0,2,0,0,0,...,1,0,0,0,1,0,0,0,0,0
10483,0,0,0,0,1,0,0,0,1,0,...,0,1,1,1,0,0,0,0,0,0


In [26]:
df_mergedFinal.to_csv('modelo1.csv', index=False)

In [20]:
tv = TfidfVectorizer(min_df=0.1, max_df=0.9, use_idf=True)
tv_matrix = tv.fit_transform(df_merged['body'])
tv_matrix = tv_matrix.toarray()

In [21]:
vocabulario = tv.get_feature_names()
df_tv = pd.DataFrame(np.round(tv_matrix, 2), columns=vocabulario)
df_tv.sample(n=5)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10,2000,2002,also,best,business,cc,click,com,company,...,thanks,time,today,us,use,want,way,well,work,would
7099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.13,0.67,0.0,...,0.0,0.11,0.0,0.11,0.0,0.0,0.0,0.0,0.0,0.11
5185,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.4,0.0,0.0,0.00
11098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.31,0.0,...,0.0,0.00,0.0,0.31,0.0,0.0,0.0,0.0,0.0,0.00
7367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,...,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00
637,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.00,0.0,...,0.0,0.33,0.0,0.35,0.0,0.0,0.0,0.0,0.0,0.00


In [22]:
df_tv.to_csv('modelo2.csv', index=False)

In [23]:
bv = CountVectorizer(ngram_range=(1,2), min_df=0.1, max_df=0.9)
bv_matrix = bv.fit_transform(df_merged['body'])
bv_matrix = bv_matrix.toarray()
vocabulario = bv.get_feature_names()
df_bv = pd.DataFrame(bv_matrix, columns=vocabulario)

df_bv.sample(n=5)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10,2000,2002,also,best,business,cc,click,com,company,...,thanks,time,today,us,use,want,way,well,work,would
7202,0,0,1,4,0,3,0,0,0,5,...,0,7,0,0,1,0,0,1,1,0
2825,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
3065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9468,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8516,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
df_bv.to_csv('modelo3.csv', index=False)

# Parte 2 - Implementación del modelo

## Separación de datos

In [27]:
df_01 = pd.read_csv('modelo1.csv')
df_02 = pd.read_csv('modelo2.csv')
df_03 = pd.read_csv('modelo3.csv')

In [28]:
bow_train, bow_test, bow_target_train, bow_target_test = model_selection.train_test_split(df_01, df_merged['status'], test_size=0.3, random_state=314)

In [29]:
bigram_train, bigram_test, bigram_target_train, bigram_target_test = model_selection.train_test_split(df_02, df_merged['status'], test_size=0.3, random_state=314)

In [30]:
tf_id_train, tf_id_test, tf_id_target_train, tf_id_target_test = model_selection.train_test_split(df_03, df_merged['status'], test_size=0.3, random_state=314)

## Implementación

In [31]:
# Modelo 1
model1 = MultinomialNB()
model1.fit(bow_train, bow_target_train)

pred_bow = model1.predict(bow_test)

print(metrics.accuracy_score(bow_target_test, pred_bow))
print('Matriz de confusion', metrics.confusion_matrix(bow_target_test, pred_bow))
print(metrics.classification_report(bow_target_test, pred_bow, target_names=['0', '1']))


0.7864561695056086
Matriz de confusion [[2196  564]
 [ 464 1590]]
              precision    recall  f1-score   support

           0       0.83      0.80      0.81      2760
           1       0.74      0.77      0.76      2054

    accuracy                           0.79      4814
   macro avg       0.78      0.78      0.78      4814
weighted avg       0.79      0.79      0.79      4814



In [32]:
metrics.plot_roc_curve(model1, bow_test, bow_target_test)
plt.show()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_roc_curve is deprecated; Function :func:`plot_roc_curve` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: :meth:`sklearn.metric.RocCurveDisplay.from_predictions` or :meth:`sklearn.metric.RocCurveDisplay.from_estimator`.
  warnings.warn(msg, category=FutureWarning)


In [33]:
#Modelo 2
modelo2 = MultinomialNB()
modelo2.fit(bigram_train, bigram_target_train)

pred_bow = modelo2.predict(bow_test)

print(metrics.accuracy_score(bow_target_test, pred_bow))
print('Matriz de confusion', metrics.confusion_matrix(bow_target_test, pred_bow))
print(metrics.classification_report(bow_target_test, pred_bow, target_names=['0', '1']))

0.8415039468217699
Matriz de confusion [[2303  457]
 [ 306 1748]]
              precision    recall  f1-score   support

           0       0.88      0.83      0.86      2760
           1       0.79      0.85      0.82      2054

    accuracy                           0.84      4814
   macro avg       0.84      0.84      0.84      4814
weighted avg       0.84      0.84      0.84      4814



In [34]:
metrics.plot_roc_curve(modelo2, bigram_test, bigram_target_test)
plt.show()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_roc_curve is deprecated; Function :func:`plot_roc_curve` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: :meth:`sklearn.metric.RocCurveDisplay.from_predictions` or :meth:`sklearn.metric.RocCurveDisplay.from_estimator`.
  warnings.warn(msg, category=FutureWarning)


In [35]:
# Modelo 3
modelo3 = MultinomialNB()
modelo3.fit(bigram_train, bigram_target_train)

pred_bow = modelo3.predict(bow_test)

print(metrics.accuracy_score(bow_target_test, pred_bow))
print('Matriz de confusion', metrics.confusion_matrix(bow_target_test, pred_bow))
print(metrics.classification_report(bow_target_test, pred_bow, target_names=['0', '1']))

0.8415039468217699
Matriz de confusion [[2303  457]
 [ 306 1748]]
              precision    recall  f1-score   support

           0       0.88      0.83      0.86      2760
           1       0.79      0.85      0.82      2054

    accuracy                           0.84      4814
   macro avg       0.84      0.84      0.84      4814
weighted avg       0.84      0.84      0.84      4814



In [36]:
metrics.plot_roc_curve(modelo3, tf_id_test, tf_id_target_test)
plt.show()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function plot_roc_curve is deprecated; Function :func:`plot_roc_curve` is deprecated in 1.0 and will be removed in 1.2. Use one of the class methods: :meth:`sklearn.metric.RocCurveDisplay.from_predictions` or :meth:`sklearn.metric.RocCurveDisplay.from_estimator`.
  warnings.warn(msg, category=FutureWarning)


# Conclusión

Dados los resultados, los modelos más confiables son el 2 y el 3, es decir el modelo de bi-gramas y TF-IDF, ya que ambos cuenta con una precisión del 84%, que si bien no está tan mal, este podría ser mucho más preciso. Esto quiere decir que ambos modelos van a clasificar si un correo es spam correctamente el 84% de las veces aproximadamente. Si bien estos modelos podrían mejorar su precisión, el cometer errores no presenta un mayor riesgo para los usuarios, pues el SPAM la mayoría de las veces solamente es publicidad, en otras palabras, si al usuario le llega un correo de SPAM no va a ser el fin del mundo, no va a tener complicaciones.